In [82]:
import torch
import torch.nn as nn
import re

In [83]:
batch_size = 128 
block_size = 256 
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 4
n_layer = 2
dropout_rate = 0.4

In [84]:
with open('tinyshakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read() 

# Add space around specified punctuation symbols
processed_text = re.sub(r'([,.:!?$&\-;(){}\[\]])', r' \1 ', text)

# Replace multiple spaces with a single space, but don't touch newlines
processed_text = re.sub(r'[^\S\n]+', ' ', processed_text).strip()

# Ensure no spaces before or after newlines in one step
processed_text = re.sub(r'\s*\n\s*', '\n', processed_text)

# vocab = sorted(list(set(processed_text.split(' '))))
vocab = sorted(list(set(re.split(r'(\s+)', processed_text))))

wtoi = {w:i for i,w in enumerate(vocab)}
itow = {i:w for i,w in enumerate(vocab)}

encode = lambda words: [wtoi[word] for word in words]
decode = lambda codes: [itow[code] for code in codes]

print(encode('I am false !'.split(' ')))
print(decode(encode('I am false !'.split(' '))))

[1538, 3493, 6729, 2]
['I', 'am', 'false', '!']


In [85]:
vocab_size = len(vocab)
print('n vocab', vocab_size)

n vocab 14566


In [86]:
print(processed_text[:2000])

First Citizen :
Before we proceed any further , hear me speak .
All :
Speak , speak .
First Citizen :
You are all resolved rather to die than to famish ?
All :
Resolved . resolved .
First Citizen :
First , you know Caius Marcius is chief enemy to the people .
All :
We know't , we know't .
First Citizen :
Let us kill him , and we'll have corn at our own price .
Is't a verdict ?
All :
No more talking on't ; let it be done : away , away !
Second Citizen :
One word , good citizens .
First Citizen :
We are accounted poor citizens , the patricians good .
What authority surfeits on would relieve us : if they
would yield us but the superfluity , while it were
wholesome , we might guess they relieved us humanely ;
but they think we are too dear : the leanness that
afflicts us , the object of our misery , is as an
inventory to particularise their abundance ; our
sufferance is a gain to them Let us revenge this with
our pikes , ere we become rakes : for the gods know I
speak this in hunger for br

In [87]:
data = torch.tensor(encode(re.split(r'(\s+)', processed_text)))
print(data.shape)

n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

torch.Size([504597])


In [88]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    start_i = torch.randint(len(data) - block_size, (32, ))
    x = torch.stack([data[i:i+block_size] for i in start_i])
    y = torch.stack([data[i+1:i+1+block_size] for i in start_i])
    x, y = x.to(device), y.to(device)

    return x,y

xb, yb = get_batch('train')
print(xb[0].shape)

b = 0 
# for t in range(block_size):
for t in range(10):
    print(decode(xb[b, :t+1].tolist()), '-->', decode(yb[b, t].unsqueeze(0).tolist()))

torch.Size([256])
['mayst'] --> [' ']
['mayst', ' '] --> ['thou']
['mayst', ' ', 'thou'] --> [' ']
['mayst', ' ', 'thou', ' '] --> ['live']
['mayst', ' ', 'thou', ' ', 'live'] --> [' ']
['mayst', ' ', 'thou', ' ', 'live', ' '] --> ['in']
['mayst', ' ', 'thou', ' ', 'live', ' ', 'in'] --> [' ']
['mayst', ' ', 'thou', ' ', 'live', ' ', 'in', ' '] --> ["Richard's"]
['mayst', ' ', 'thou', ' ', 'live', ' ', 'in', ' ', "Richard's"] --> [' ']
['mayst', ' ', 'thou', ' ', 'live', ' ', 'in', ' ', "Richard's", ' '] --> ['seat']


In [89]:
from torch.nn import functional as F
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__ ()
        self.key = nn.Linear(n_embd, head_size, bias=False) 
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones((block_size, block_size))))

        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        B, T, C = x.shape # BATCH_SIZE, BLOCK_SIZE, EMBD_SIZE
        k = self.key(x)
        q = self.query(x)
        
        attn = q @ k.transpose(-2, -1) * C**-0.5 # (B,T,C) @ (B, C, T) --> (B, T, T)
        attn = attn.masked_fill(self.tril[:T, :T] == 0, float('-inf')) 
        attn = F.softmax(attn, dim=-1)
        attn = self.dropout(attn)

        v = self.value(x)
        out = attn @ v # (B,T,T) @ (B,T,C) --> (B,T,C)
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out
    
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout_rate),
        )
    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    def __init__(self, n_embd, n_heads):
        super().__init__()
        head_size = n_embd//n_heads
        self.sa = MultiHeadAttention(num_heads=n_heads, head_size=head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x
    
class Model(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embd_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd=n_embd, n_heads=n_head) for _ in range(n_layer)])
        self.ln = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)


    def forward(self, tokens, targets = None):
        B,T = tokens.shape

        tok_emb = self.token_embd_table(tokens) # (BATCH_SIZE,VOCAB_SIZE,EMBD_SIZE) B,T,C
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln(x)
        logits = self.lm_head(x) # (BATCH_SIZE, EMBD_SIZE, VOCAB_SIZE)

        if targets == None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, tokens, max_len):
        for _ in range(max_len):
            # logits, loss = self(tokens)
            logits, loss = self(tokens[:, -block_size:])
            logits = logits[:, -1, :] # (BATCH_SIZE, EMBD_SIZE)
            probs = F.softmax(logits, dim=-1)
            token_next = torch.multinomial( probs, num_samples=1)
            tokens = torch.cat((tokens, token_next), dim=1) # (BATCH_SIZE, TOKEN_SIZE+1)

        return tokens


In [90]:
print(device)
m = Model(vocab_size=vocab_size)
m = m.to(device)

cuda


In [91]:
decoded = decode(m.generate(tokens=torch.zeros((1,1), dtype=torch.long, device=device), max_len=100)[0].tolist())
print(''.join(decoded))


apeharbourdevicescaeteraconsciencemeedslenityheard'stmothsactorsBelgialitteredwantingCitizenstitlesdistinguishheirspowerfulDestroy'dUnlawfullyTearabsencepernicioustiger'skinsman'sbrinishconcupisciblefornicatresslordstransgressionhairymournersjoinriddlingbonfiresbentshunsCogmakeRebell'dparcelssociablematterRICHARDcupbearerwishearthlyGraybeardenjoyingMortalembracedinfantdemesnesBearaptreekBUCKINGHAMsteal'GrownElbowpeasantpuissancesuck'dstpilotoftHoverthriveenroll'd cockleLondonCalibansSirrahusurp'dLucentiomessagegrantCountingsecretacclamationsyoung'stlamentedContenttyrannymissinglyhouseholds'Hope'Gainstsmileshownbawcockmouthshelveskillseitherroaredshakenlearnsexceptionswinning


In [92]:
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [93]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)
for iter in range(max_iters):
    # print(iter)
    if (iter+1)%eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter+1}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 500: train loss 3.2204, val loss 3.3573
step 1000: train loss 3.1102, val loss 3.3041
step 1500: train loss 2.8351, val loss 3.1218
step 2000: train loss 2.6151, val loss 3.0434
step 2500: train loss 2.4663, val loss 3.0318
step 3000: train loss 2.3501, val loss 3.0478
step 3500: train loss 2.2379, val loss 3.0760
step 4000: train loss 2.1231, val loss 3.1219
step 4500: train loss 2.0345, val loss 3.1737
step 5000: train loss 1.9471, val loss 3.2376


In [ ]:

input = torch.tensor(0, dtype=torch.long, device=device)
input = torch.unsqueeze(input, 0)
print(''.join(decode(m.generate(tokens=input, max_len=200)[0].tolist())))

I was killed by the farthest given with her head
With artificial vineyard . Beshrew him ,
And not so cunning - doing thee ,
To scarlet base vows .
CLIFFORD :
Then he will escape be better , like toys out of mind ,
And back me with whom thy napkin usurping mouth
To upstart venom up than my death ,
Be with flames like , even in himself ,
And give me from the impossibilities , ,
To fear a innocent hap and faithful with a sin ,
And spies them to her , sick , too .
Thy steps gentle father
